In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import networkx as nx
import json
import sys
import os
from networkx.readwrite import json_graph
sys.path.insert(0, os.path.abspath('..'))
import  parser.matcher as matcher
import blazegraph_querier.network_analysis as network_analysis

In [3]:
graph = nx.DiGraph()

In [7]:
def load_sigma_file(filename):
    with open(filename, 'r') as jsonfile:
        graph_json = json.load(jsonfile)

    nodes = graph_json['nodes']
    edges = graph_json['edges']
    graph = network_analysis.get_network(nodes, edges)
    return nodes, graph

In [8]:
rootpath = '/media/sf_VBox_Shared/CaseLaw/graphs/json_sigma/'
filename = 'sigma_insolventierecht.json'
nodes, graph = load_sigma_file(os.path.join(rootpath, filename))

In [9]:
graph.size()

44

In [54]:
for node in nodes:
    article_str = ', '.join(node['articles'].keys())
    if article_str == '':
        article_str = 'none'
    node['article'] = article_str

In [55]:
nodes[0]

{'abstract': 'WSNP. Beëindiging schuldsaneringsregeling zonder schone lei (81 RO).',
 'article': 'none',
 'articles': {},
 'count_annotation': 0,
 'count_version': 2,
 'creator': 'http://standaarden.overheid.nl/owms/terms/Hoge_Raad_der_Nederlanden',
 'date': '2009-11-27',
 'id': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2009:BJ9941',
 'subject': 'http://psi.rechtspraak.nl/rechtsgebied#civielRecht',
 'title': 'ECLI:NL:HR:2009:BJ9941 Hoge Raad , 27-11-2009 / 09/01158'}

In [6]:
graph.nodes()[0]

'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2016:998'

In [56]:
hubs, authorities = nx.hits(graph)
statistics = {
    'degree' : nx.degree(graph),
    'in_degree' : graph.in_degree(),
    'out_degree' : graph.out_degree(),

    'degree_centrality' : nx.degree_centrality(graph),
    'in_degree_centrality' : nx.in_degree_centrality(graph),
    'out_degree_centrality' : nx.out_degree_centrality(graph),
    'betweenness_centrality' : nx.betweenness_centrality(graph),
    'hubs': hubs,
    'authorities': authorities
}

In [57]:
def url_to_ecli(url):
    return url.split('=')[-1]

In [58]:
result = []
statistics_variables = [ 'references', 'count_version', 'count_annotation', 'article', 'date', 'year', 'subject', 'creator',
                        'degree', 'in_degree', 'out_degree', 
                        'degree_centrality','in_degree_centrality', 'out_degree_centrality', 'betweenness_centrality',
                        'hubs', 'authorities'
                      ]

for node in nodes:
    nodeid = node['id']
    
    atts = {var: statistics[var][nodeid] for var in statistics.keys()}
    atts['date'] = node.get('date', '')
    atts['year'] = int(url_to_ecli(nodeid).split(':')[3])
    atts['subject'] = node['subject']
    atts['creator'] = node['creator']
    atts['article'] = node['article']
    atts['count_version'] = node['count_version']
    atts['count_annotation'] = node['count_annotation']
    atts['references'] = [url_to_ecli(url) for url in graph[nodeid].keys() ]
    atts['id'] = url_to_ecli(nodeid)
    result.append(atts)

In [59]:
import pandas as pd
result_df = pd.DataFrame(result)
result_df = result_df.set_index('id')
result_df = result_df[statistics_variables]
result_df.head()

,references,count_version,count_annotation,article,date,year,subject,creator,degree,in_degree,out_degree,degree_centrality,in_degree_centrality,out_degree_centrality,betweenness_centrality,hubs,authorities
id,,,,,,,,,,,,,,,,,
ECLI:NL:HR:2009:BJ9941,[],2,0,none,2009-11-27,2009,http://psi.rechtspraak.nl/rechtsgebied#civielR...,http://standaarden.overheid.nl/owms/terms/Hoge...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
ECLI:NL:HR:2009:BJ7317,[],2,0,none,2009-10-16,2009,http://psi.rechtspraak.nl/rechtsgebied#civielR...,http://standaarden.overheid.nl/owms/terms/Hoge...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
ECLI:NL:HR:2009:BJ7535,[],2,0,none,2009-10-23,2009,http://psi.rechtspraak.nl/rechtsgebied#civielR...,http://standaarden.overheid.nl/owms/terms/Hoge...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
ECLI:NL:HR:2009:BJ7835,[],2,0,none,2009-10-23,2009,http://psi.rechtspraak.nl/rechtsgebied#civielR...,http://standaarden.overheid.nl/owms/terms/Hoge...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
ECLI:NL:HR:2009:BJ8339,[],2,0,none,2009-11-13,2009,http://psi.rechtspraak.nl/rechtsgebied#civielR...,http://standaarden.overheid.nl/owms/terms/Hoge...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
outfile = os.path.join('/media/sf_VBox_Shared/CaseLaw/spot/', 'spot_'+filename[6:-4]+'csv')
result_df.to_csv(outfile)

In [61]:
outfile = os.path.join('/media/sf_VBox_Shared/CaseLaw/spot/', 'spot_'+filename[6:-4]+'json')
with open(outfile, 'w') as fp:
    json.dump(result, fp)